In [1]:
#importing necessary libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

import dash 
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
import numpy as np
from dash_table import DataTable

/var/folders/f8/857c9fm55pl4nmhsj_93hykm0000gn/T/ipykernel_3455/4085926308.py:10: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/f8/857c9fm55pl4nmhsj_93hykm0000gn/T/ipykernel_3455/4085926308.py:11: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/f8/857c9fm55pl4nmhsj_93hykm0000gn/T/ipykernel_3455/4085926308.py:16: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  from dash_table import DataTable


In [2]:
#importing necessary excel and csv
df=pd.read_csv('Crime_Data_semi_cleaned_2021.csv')
ethnicity_lookup = pd.read_excel('Crime data dictionary.xlsx',sheet_name='Sheet3')

In [3]:
#merging between CSVs to get the ethnicity
df = df.merge(ethnicity_lookup, left_on='Vict Descent', right_on='Code', how='left')

In [5]:
#Separating Year, Date and Month in a separate column
df['DATE OCC'] = pd.to_datetime(df['DATE OCC']) #may not be required if already done before
df['YEAR OCC'] = df['DATE OCC'].dt.year
df['MONTH OCC'] = df['DATE OCC'].dt.month
df=df.drop_duplicates(subset=['DR_NO'])
# df.head()

In [6]:
#Fillig NA gender and the ones apart from Female and Male as Unknown  
df['Vict Sex'].fillna('Unknown',inplace=True)
df["Vict Sex"] = df["Vict Sex"].map({"X" : "Unknown",
                     "H" : "Unknown",
                     "N" : "Unknown",
                     "M" : "M",
                     "F" : "F",
                     "Unknown" : "Unknown"})
df['Crime Type'] = df['Part 1-2'].map({1 : "Severe Crime",
                     2 : "Non-Severe Crime"})

In [7]:
#Dropping unnecessary columns
df.drop(columns=['Unnamed: 0','Crm Cd','Premis Cd','Weapon Used Cd','Status','Cross Street'],inplace=True)
df.reset_index(inplace=True)
df.drop(columns=['index'],inplace=True)

#Getting time by hour to ge the time of the day

df['TIME OCC HR'] = pd.to_datetime(df['TIME OCC']).dt.hour
#may need to change this
df['Age Group'] = pd.cut(df['Vict Age'], [0,25,40,60,100],labels = ['0-25','25-40','40-60','60-100']).astype(str).fillna("Unknown")
# df1['TIME OCC HR'] = df1['TIME OCC'].apply(lambda x: x.hour) #check type of time occ if this does not work
df['Time of Day'] = df['TIME OCC HR'].apply(lambda x : 'Morning' if 4<=x<12 else 'Afternoon' if 12<=x<16 else 'Evening' if 16<=x<20 else 'Night' )


In [8]:
#Assigning Status as Open and Closed
df["Status"] = df["Status Desc"].map({"Invest Cont" : "open",
                     "Adult Other" : "closed",
                     "Adult Arrest" : "closed",
                     "Juv Arrest" : "closed",
                     "Juv Other" : "closed",
                     "UNK" : "closed"})

In [9]:
#App code

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SIMPLEX])

    
area_names= df['AREA NAME'].unique()

options = [
    {"label": " Male", "value": "M"},
    {"label": " Female", "value": "F"},
]


genders = [option["value"] for option in options]



#App layout
app.layout = html.Div([
    
    #Heading
    html.Div([
        html.H1("Crime in LA County : An Analysis")        
    ]  
        , style={'text-align':'center',"font-family":"Cursive",'margin-top':'auto'}
    ),

    #HTML heat-map    
        html.Div([
            html.Iframe(srcDoc=open('./LA_Crime_map.html', 'r').read(), id='heatmap',style={"height": "400px", "width": "100%",'margin':'auto','display': 'block'})        
        ]  , style={'width': '80%', 'display': 'block','margin': 'auto','margin-top':'20px','margin-bottom':'10px','box-shadow': '0px 0px 8px lightgrey'}),
    
    #Dropdown 
    html.Div([dcc.Dropdown(
            id='area',
            options=[{'label': i, 'value': i} for i in area_names],
            value='Southwest'
        ),], style={'padding':'10px','margin-left':'auto','margin-right':'auto' ,'margin': '10px','width': '20%', 'display': 'block'}),
    
    html.Hr(style={'color':'#B1B1B1'}),
    
    #heading for table
    html.Div([
        html.H5("Most Used Weapons")        
    ]  
        , style={'text-align':'center',"font-family":"monospace",'margin-top':'5px'}
    ),
    html.Div(id = 'area-table',style = {'width':'80%','margin':'auto','margin-right':'auto','margin-left':'auto','margin-bottom':'20px',"font-family":"monospace",}),
    
        
    #Crime over a Decade and the checkboxes       
    html.Div([
        
        html.Div([
            
           
            html.Div([dcc.Checklist(
            id="total-checklist",
            options=[{"label": " Total", "value": "Total"}],
            value=[],
#             labelStyle={"display": "inline-block"},
        ),],style={'margin':'5px','display':'inline-block'}),
            
            html.Div([dcc.Checklist(
                id="gender-checklist",
                options=options,
                value=[],
                labelStyle={"display": "inline-block"},
            ),],style={'margin':'5px','display':'inline-block'}),
            
            html.Div([html.H5("Crime Over a Decade")],style={'display':'inline-block','text-align':'center','margin-left':'265px',"font-family":"monospace"}),
            
           
            #line chart
            html.Div([dcc.Loading(
                id='loading-line',
                type='circle',
                children=dcc.Graph(id='line-chart')
            )])
        ], style={'background-color':'white','width':'81.5%','display': 'block','padding':'10px' ,'margin-left': 'auto','margin-right': 'auto','box-shadow': '0px 0px 8px lightgrey'}),

    ]),
    
    #Ethnicity Graph
    html.Div([
        html.Div([
        dcc.Loading(
            id='loading-graph1-1',
            type='circle',
            children=dcc.Graph(id='left-most-graph')
        ),

    ], style={'background-color':'white','height':'920px','float': 'left','margin': '10px','box-shadow': '0px 0px 8px lightgrey','padding':'10px','width':'40%'}),

    #Par cat and Case count by Age across gender charts combined div 
    html.Div([
        #first chart div
        html.Div([
        dcc.Loading(
            id='loading-graph1',
            type='circle',
            children=dcc.Graph(id='left-graph')
        ),

    ], style={'background-color':'white','margin': '10px','box-shadow': '0px 0px 8px lightgrey','padding':'10px'}),
    
    #second chart div
    html.Div([

        dcc.Loading(
            id='loading-graph2',
            type='circle',
            children=dcc.Graph(id='middle-graph')
        )
    ], style={'background-color':'white','margin-right':'10px','margin-left': '10px','box-shadow': '0px 0px 8px lightgrey','padding':'10px'})
     ],style = {'display':'inline-block','width':'40%'}),
        
    #KPI divs
      html.Div([
        html.Div([
            dcc.Loading(
                id='kpi-1-loading',
                type='circle',
                children=dbc.Card(id='kpi-1',style={'border': 'none'})
            )],style={'padding-top':'32px','background-color':'white','text-align':'center','margin-bottom':'50px','padding':'10px' ,'box-shadow': '0px 0px 8px rgb(133, 176, 171)','width': '300px','height':'100px'}),
        html.Div([
            dcc.Loading(
                id='kpi-2-loading',
                type='circle',
                children=dbc.Card(id='kpi-2',style={'border': 'none'})
            )],style={'padding-top':'32px','background-color':'white','text-align':'center','margin-bottom':'50px','padding':'10px' ,'box-shadow': '0px 0px 8px rgb(133, 176, 171)','width': '300px','height':'100px'}),
        html.Div([
            dcc.Loading(
                id='kpi-3-loading',
                type='circle',
                children=dbc.Card(id='kpi-3',style={'border': 'none'})
            )],style={'padding-top':'32px','background-color':'white','text-align':'center','padding':'10px' ,'display':'block','box-shadow': '0px 0px 8px rgb(133, 176, 171)','width': '300px','height':'100px'}),
        
        ],style={'display':'inline-block','width': '10%','margin-top':'22%','margin-bottom':'20%','margin-left':'10px'}),
        
    ],style = {'display':'flex', 'width':'90%','margin-left':'2%'}),
    
    #Slider 
    html.Div([
    dcc.Slider(
        id='year-slider',
        min=2010,
        max=2021,
        step=1,
        value=2020,
        marks={str(year): str(year) for year in range(2010,2022)}
    ),
    html.Div(id='slider-output-container')
    ],style={'background-color':'white','display': 'block','box-shadow': '0px 0px 8px lightgrey',
             'padding':'10px','margin':'20px','margin-left':'2.85%','margin-right':'2.85%','margin-bottom':'8%','margin-top':'5px'}),
    
])


#table
@app.callback(
    Output("area-table","children"),
    [Input("area","value")]
)
def update_table(area):
    cnt_weapons = pd.DataFrame(df[df['AREA NAME']==area].groupby(['Weapon Desc'])['DR_NO'].count().sort_values(ascending=False))
    cnt_2 = cnt_weapons[0:5]
    cnt_3 = cnt_2.rename(columns = {'DR_NO':'Count'}).transpose()
    
    data = cnt_3.to_dict('rows')
    columns =  [{"name": i, "id": i,} for i in (cnt_3.columns)]
    
    dataTableRet = DataTable(
    style_header={
        'whiteSpace': 'normal',
        'width':'5px'},
    data=data,
    columns=columns,
    style_cell={'textAlign': 'right','font-family':'sans-serif','fontSize':12},)
    
    return dataTableRet


#KPI 1
@app.callback(
    [Output("kpi-1","children")],
    [Input("area","value"),
    Input('year-slider', 'value')]
)
def kpi1(area,year):
    if area == "Select Area":
        raise PreventUpdate
    
    df_new = df[(df['AREA NAME']==area)&(df['YEAR OCC']==year)]
    res= round(df_new.Status.value_counts(normalize=True)['closed']*100,1)
    text1=f"Case Resolution Rate: {res}%"

    text= [html.P(html.B(text1))]
    
    return text


#KPI 2
@app.callback(
    [Output("kpi-2","children")],
    [Input("area","value"),
    Input('year-slider', 'value')]
)
def kpi2(area,year):
    if area == "Select Area":
        raise PreventUpdate
    df_new = df[(df['AREA NAME']==area)&(df['YEAR OCC']==year)]
    
    age=int(np.ceil(df_new['Vict Age'].mean()))
    
    text1=f"Average Victim Age: {age}"
    
    text= [html.P(html.B(text1))]
    
    return text
 

#KPI 3
@app.callback(
    [Output("kpi-3","children")],
    [Input("area","value"),
    Input('year-slider', 'value')]
)
def kpi3(area,year):
    if area == "Select Area":
        raise PreventUpdate
    
    df_new = df[(df['AREA NAME']==area)&(df['YEAR OCC']==year)]

    eth=df_new['Value'].value_counts(normalize=True).head(1).index[0].replace('\xa0', ' ')
    per=round(df_new['Value'].value_counts(normalize=True).head(1).values[0]*100,1)

    text1=f"  Most Vulnerable Ethnicity: {eth}"   

    text= [
    html.P(html.B(text1)),
    ]
    
    return text    
    
    

#Checklist
@app.callback(
    [Output("gender-checklist", "value"),
    Output("total-checklist", "value")],
    [Input("gender-checklist", "value"),
    Input("total-checklist", "value")]
)
def sync_checklists(gender_selected, total_selected):
    ctx = dash.callback_context
    input_id = ctx.triggered[0]["prop_id"].split(".")[0]
    if input_id == "gender-checklist":
        total_selected = [] if bool(set(gender_selected).intersection(set(genders))) else ["Total"]
    else:
        gender_selected =  [] if total_selected else genders
    return gender_selected, total_selected



#Cases over Decade 
@app.callback(
    Output("line-chart", "figure"), 
    [Input("gender-checklist", "value"),
    Input("area","value")])
def update_line_chart(inputs,area):
    if inputs is None:
        raise PreventUpdate
    
    dfArea = df[df['AREA NAME']==area]
    dfAreaYear = dfArea.groupby(['Vict Sex','YEAR OCC'])['DR_NO'].count().reset_index().rename(columns = {'DR_NO':'Count'})

    if set(inputs) == set(['M','F']):
        mask = dfAreaYear[dfAreaYear['Vict Sex'].isin(inputs)]
        fig = px.line(mask, 
        x ='YEAR OCC', y='Count',color = 'Vict Sex')

        fig['data'][0]['line']['color']='rgb(133, 176, 171)'
        fig['data'][1]['line']['color']='rgb(31, 38, 42)'
    elif (set(inputs) == set(['M'])) or (set(inputs) == set(['F'])):
        mask = dfAreaYear[dfAreaYear['Vict Sex'].isin(inputs)]
        fig = px.line(mask, 
        x ='YEAR OCC', y='Count')
        fig['data'][0]['line']['color']='rgb(133, 176, 171)'

    else:
        mask = dfAreaYear.groupby(['YEAR OCC']).Count.sum().reset_index()
        fig = px.line(mask, 
        x ='YEAR OCC', y='Count')
        fig['data'][0]['line']['color']='rgb(31, 38, 42)'
       
    fig.update_layout(
        template='plotly_white',
        plot_bgcolor = '#fff',
        font=dict(
        family="Courier New, monospace",

        ),
        xaxis = dict(
            title='Years',
            tickmode = 'array',
            tickvals = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
            ticktext = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020','2021'],

        ),
            yaxis = dict(
            title='Case Count'
            ),

        )    
    return fig


#Ethnicity
@app.callback(
    Output("left-most-graph", "figure"), 
    [Input("area","value"),
    Input('year-slider', 'value')])
def update_left_most_graph(area,year):
    df2 = df[(df['AREA NAME']==area)&(df['YEAR OCC']==year)]
    fig = px.histogram(df2, 
                   y="Value"
                   ,color='Crime Type',
                   color_discrete_map = {'Severe Crime':'rgb(133, 176, 171)','Non-Severe Crime':'rgb(52, 53, 52)'}
                   ,labels={'Value':'Victim Descent','Crime Type':'Type'}
                   ,opacity=0.95, height = 800,                   
                  ).update_yaxes(categoryorder='total ascending')



    fig.update_layout(
        font=dict(
            family="Courier New, monospace",),
        title_text='Case Count by Victim Ethnicity', # title of plot
        yaxis_title_text='', # yaxis label
        xaxis_title_text='', # yaxis label
        bargap=0.1, # gap between bars of adjacent location coordinates
        plot_bgcolor="White",
        paper_bgcolor="White",
        legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.05,
            xanchor="right",
            x=0.80
        ),
        legend_title = dict(font = dict(size = 12)),
        annotations=[
        go.layout.Annotation(
        {
                'showarrow': False,
            'text': '',
            'x': 0.5,
            'xanchor': 'center',
            'xref': 'paper',
            'y': 0,
            'yanchor': 'top',
            'yref': 'paper',
            'yshift': -10,

        })
    ]
        
    )
    
    return fig


#Case count by age across gender
@app.callback(
    Output("left-graph", "figure"), 
    [Input("area","value"),
    Input('year-slider', 'value')])
def update_left_graph(area,year):
    dfArea = df[df['AREA NAME']==area]
    dfAreaYear = dfArea.groupby(['Vict Sex','YEAR OCC'])['DR_NO'].count().reset_index().rename(columns = {'DR_NO':'Count'})

    if dfAreaYear['Count'].sum() == 0:
        raise PreventUpdate
        
    fig=go.Figure()
    #Filter the data
    df_g=df[(df['YEAR OCC']==year) & (df['AREA NAME']==area) & df['Vict Sex'].isin(['M','F'])]
    
    if dfAreaYear['Count'].sum() != 0:
        #Group for Age, 
        graph1=pd.DataFrame(df_g.groupby(['Vict Age','Vict Sex','Crime Type'])['DR_NO'].agg('count'))
        graph1=graph1.reset_index()
        fig = px.line(graph1, x='Vict Age',y='DR_NO', facet_col='Vict Sex',color='Crime Type')
        fig.update_yaxes(showgrid=True)
        fig.update_xaxes(title_text='Male Age',showgrid=True)
        fig.update_xaxes(rangeselector_xanchor='right')
        fig['data'][2]['line']['color']='rgb(133, 176, 171)'
        fig['data'][1]['line']['color']='rgb(52, 53, 52)'
        fig['data'][3]['line']['color']='rgb(133, 176, 171)'
        fig['data'][0]['line']['color']='rgb(52, 53, 52)'
        fig.update_layout(
            
        template='plotly_white',
        title="Case Count by Age across Gender",
#             xaxis=dict(anchor='right'),
            xaxis_title=dict(text='Female Age'),
            yaxis_title="",
            font=dict(
            family="Courier New, monospace",),
                
            legend=dict(
                orientation="h",
                yanchor="top",
                y=-0.15,
                xanchor="right",
                x=0.60
            ),
            legend_title = dict(font = dict(size = 12)),
            annotations=[
            go.layout.Annotation(
            {
#                 'showarrow': False,
                'text': '',
                'x': 0.1,
                'xanchor': 'right',
                'xref': 'paper',
                'y': 0,
                'yanchor': 'top',
                'yref': 'paper',
                'yshift': -10,

            })
        ]

    
        )
        return fig


#par-cat 
@app.callback(
    Output("middle-graph", "figure"), 
    [Input("area","value"),
    Input('year-slider', 'value')])
def update_right_graph(area,year):
    
    #Filter the data
    df_g1=df[(df['YEAR OCC']==year) & (df['AREA NAME']==area)]

    color = df_g1['Part 1-2']
    colorscale = [[0, 'rgb(133, 176, 171)'],[1, 'rgb(52, 53, 52)']]; #can change color scheme
    fig = go.Figure(data = [go.Parcats(dimensions=[{'label': i, 'values':df_g1[i]} for i in ['Time of Day','Age Group','Vict Sex','Crime Type']],
            line={'color': color
                  , 'colorscale': colorscale
                 })])

    fig.update_layout(
        title="Time of Day across Victim Demographics",
        
        height=420,
        width=480,
#         template='plotly_white',
        font=dict(
        family="Courier New, monospace",
#         color="#7f7f7f"
        )
    )
    return fig


app.run_server(debug = True)

Dash app running on http://127.0.0.1:8050/


/var/folders/f8/857c9fm55pl4nmhsj_93hykm0000gn/T/ipykernel_3455/925219120.py:170: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

